In [1]:
import pandas as pd
from pandas.io.json import json_normalize
import os
import json

import numpy as np
import re
import csv
import sys
import matplotlib.pyplot as plt
import seaborn as sns
#from urllib.parse import urlparse 


In [2]:
root_path = "/Users/wangjiabin/Desktop/User Profiling/data/impression-2018"

In [3]:
#for path,dir_list,file_list in os.walk(root_path):
#    print(path)

In [3]:
def load_data(path):
    
    df = pd.DataFrame()
    name_list=[]
    
    for path,dir_list,file_list in os.walk(path):         
        name_list.append(file_list)
        
        for file_name in file_list:
            if '.DS_Store' not in file_name.split('/')[-1]:
                df = df.append(pd.read_json(path+'/'+file_name,lines=True),ignore_index=True)
        
    meta = json_normalize(df.meta)
    univ = json_normalize(df.universal)
    df = df.join([meta,univ])
    return df

In [4]:
data = load_data(root_path)

In [5]:
data.shape

(264033, 55)

In [6]:
data.to_csv("data0607.csv")

# PROCESSING

In [7]:
train = data
#train.dtypes

In [8]:

def create_features(df,enforce_cols=None):
    
    #df = df.set_index(['MediaGammaImpression']) 
    df = df.drop(['cookie','meta','recommendation.items','universal','basket.line_items'],axis=1)
    
    
    df['screen.size'] = df['screen.height'] * df['screen.width']
    df['screen.size'] = df['screen.size'].astype(str)
    
    df['basket.item_count'] = df['basket.item_count'].replace(np.nan,'0')
    df['basket.item_count'] = df['basket.item_count'].astype(int)
    
    str_df = df.loc[:, df.dtypes == np.object].join(df.loc[:, df.dtypes == np.bool])
    sta_df = df.loc[:, df.dtypes == np.float64].join(df.loc[:, df.dtypes == np.int64])
    sta_df = sta_df.join(df['MediaGammaImpression'])
   
    str_df = str_df.replace(np.nan,'')
    str_df = str_df.astype(str)
    str_df['navigator.language'] = str_df['navigator.language'].apply(lambda x: x.upper())
    

    str_df = str_df.groupby('MediaGammaImpression').agg(lambda x: ''.join(set(x)))
    
    sta_df = df.groupby('MediaGammaImpression').agg({'basket.shipping_cost': 'sum',
                                                     'basket.subtotal': 'max',
                                                     'basket.tax': 'sum',
                                                     'basket.total': 'sum',
                                                     'product.unit_sale_price': 'sum',
                                                     'basket.item_count': 'max' })
    
    
    df = str_df.join(sta_df)
    
    df = df.reset_index(level='MediaGammaImpression')
    df = df.loc[df['MediaGammaImpression'].apply(lambda x: x != '')]
    df = df.set_index(['MediaGammaImpression']) 
    
    return str_df,sta_df,df
        


In [9]:
str_df,sta_df,group_train = create_features(train)


In [10]:
group_train.shape

(41928, 45)

In [11]:
#group_train.to_csv("group_train.csv")
group_train.to_csv("group.csv")

In [12]:
group_train.head()

,custom,server,date.date,document.lastModified,document.referrer,document.title,href,navigator.appCodeName,navigator.appName,navigator.appVersion,...,version,screen.size,initial,navigator.cookieEnabled,basket.shipping_cost,basket.subtotal,basket.tax,basket.total,product.unit_sale_price,basket.item_count
MediaGammaImpression,,,,,,,,,,,,,,,,,,,,,
00000874049d769b36451f9daf9a8ad3,{},"{'REMOTE_ADDR': '82.37.108.173, 35.241.60.129'}",2018-06-30T10:42:05.491Z2018-06-30T08:38:34.72...,06/30/2018 11:42:0906/30/2018 09:42:2306/30/20...,https://www.petspyjamas.com/travel/booking/abo...,Dog-Friendly Holidays and Pet-Friendly Travel ...,https://www.petspyjamas.com/travel/booking/abo...,Mozilla,Netscape,5.0 (iPhone; CPU iPhone OS 11_3 like Mac OS X)...,...,,181760,TrueFalse,True,NaN,NaN,NaN,NaN,NaN,0.0
0000d19da180899b509d8c4560dd9b89,{},"{'REMOTE_ADDR': '34.224.214.253, 35.241.60.129'}",2018-06-25T13:51:41.488Z,06/25/2018 13:51:41,,Welcome | PetsPyjamas,https://www.petspyjamas.com/,Mozilla,Netscape,5.0 (Macintosh; Intel Mac OS X 10_13_4) AppleW...,...,,480000,True,True,NaN,NaN,NaN,NaN,NaN,0.0
00050d2cdf8bd5c0e9f0e7aa81d20bf2,{},"{'REMOTE_ADDR': '66.249.69.98, 35.241.60.129'}",2018-07-02T00:00:00.840Z,07/01/2018 17:00:00,,"Shop Basenji, Blue, Creature Clothes, Collars,...",https://www.petspyjamas.com/pets/breed-basenji...,Mozilla,Netscape,"5.0 AppleWebKit/537.36 (KHTML, like Gecko; com...",...,,1048576,True,True,NaN,NaN,NaN,NaN,NaN,0.0
00062a5a6bf0e73f7a2c19f4683c7fcf,{},"{'REMOTE_ADDR': '86.14.8.238, 35.241.60.129'}",2018-06-26T05:15:15.273Z2018-06-26T05:14:21.389Z,06/26/2018 06:14:2106/26/2018 06:15:15,https://www.petspyjamas.com/travel/nav/dog-fri...,Dog-friendly 2 People | PetsPyjamasDog-friendl...,https://www.petspyjamas.com/travel/nav/dog-fri...,Mozilla,Netscape,5.0 (Linux; Android 8.1.0; Pixel 2 Build/OPM2....,...,,301584,TrueFalse,True,NaN,NaN,NaN,NaN,NaN,0.0
00079523ef5c0c723f455c2e54df5467,{},"{'REMOTE_ADDR': '157.37.91.210, 35.241.60.129'}",2018-06-29T18:33:57.337Z2018-06-29T18:35:10.392Z,06/30/2018 00:03:5706/30/2018 00:05:10,https://www.petspyjamas.com/,Welcome | PetsPyjamasContact Us | PetsPyjamas,https://www.petspyjamas.com/https://www.petspy...,Mozilla,Netscape,5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36...,...,,1049088,TrueFalse,True,NaN,NaN,NaN,NaN,NaN,0.0


In [13]:
def pre_process_data(df, enforce_cols=None):
    #df.fillna(0, inplace=True)
    
    df['initial'] = df['initial'].apply(lambda x: 'False' if x == 'False'
                                    else('New' if 'TrueFalse' in x 
                                         else 'True'))
    
   
    df['document.referrer'] = df['document.referrer'].apply(lambda x: 
        'both' if 'petspyjamas.com/product' in x and 'petspyjamas.com/travel' in x
                    else('travel' if 'petspyjamas.com/travel' in x and  'petspyjamas.com/product' not in x 
                         else ('product' if 'petspyjamas.com/travel' not in x and  'petspyjamas.com/product' in x 
                               else 'others')))
    
    
    #df['document.referrer'] = df['document.referrer'].apply(lambda x: 'product' if 'petspyjamas.com/product' in x 
    #                                                        else('travel' if 'petspyjamas.com/travel' in x 
    #                                                             else('pet' if 'petspyjamas.com/pet' in x 
    #                                                                  else 'others'))) 
    df['href'] = df['href'].apply(lambda x: 'product' if 'petspyjamas.com/product' in x 
                                               else('travel' if 'petspyjamas.com/travel' in x 
                                                    else('pet' if 'petspyjamas.com/pet' in x 
                                                         else 'others'))) 

    #
    #document.title
    df['document.title'] = df['document.title'].apply(lambda x: 'both' if 'Shop' in x and 'Dog-friendly' in x
                                                      else('Shop' if 'Shop' in x and  'Dog-friendly' not in x 
                                                           else ('Dog-friendly' if 'Dog-friendly' in x and  'Shop' not in x
                                                                 else 'others')))
    
    
    df['page.location'] = df['page.location'].apply(lambda x: 'no_location' if x ==''  else 'location') 

    
    #navigator.platform
    
    df['navigator.platform'] = df['navigator.platform'].apply(lambda x: 'Linux' if 'Linux' in x
                                                      else('Win' if 'Win' in x  else x))
    
    #basket.currency
    df['basket.currency'] = df['basket.currency'].apply(lambda x: 'pay' if 'GBP' in x else 'not_pay')
    
    
    #product.url
    df['product.url'] = df['product.url'].apply(lambda x: 'product' if 'product' in x else 'others')
    
    #basket.total
    df.ix[df['basket.total'].isnull(), 'baskettotal'] = 0
    df.ix[df['basket.total'].between(1, 100), 'baskettotal'] = 1
    df.ix[df['basket.total'].between(100, 250), 'baskettotal'] = 2
    df.ix[df['basket.total'].between(250, 500), 'baskettotal'] = 3
    df.ix[df['basket.total']>500, 'baskettotal'] = 3
    
    #baskettotal#
    df.ix[df['basket.subtotal'].isnull(), 'basketsubtotal'] = 0
    df.ix[df['basket.subtotal'].between(1, 15), 'basketsubtotal'] = 1
    df.ix[df['basket.subtotal'].between(15, 50), 'basketsubtotal'] = 2
    df.ix[df['basket.subtotal'].between(50, 200), 'basketsubtotal'] = 3
    df.ix[df['basket.subtotal']>200, 'basketsubtotal'] = 3

    return df

In [14]:
group_train_pre = pre_process_data(group_train)


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:49: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [15]:
group_train_pre.shape

(41928, 47)

In [17]:
group_train_pre.to_csv("group_train_final.csv")

In [16]:
group_train_pre

,custom,server,date.date,document.lastModified,document.referrer,document.title,href,navigator.appCodeName,navigator.appName,navigator.appVersion,...,initial,navigator.cookieEnabled,basket.shipping_cost,basket.subtotal,basket.tax,basket.total,product.unit_sale_price,basket.item_count,baskettotal,basketsubtotal
MediaGammaImpression,,,,,,,,,,,,,,,,,,,,,
00000874049d769b36451f9daf9a8ad3,{},"{'REMOTE_ADDR': '82.37.108.173, 35.241.60.129'}",2018-06-30T10:42:05.491Z2018-06-30T08:38:34.72...,06/30/2018 11:42:0906/30/2018 09:42:2306/30/20...,travel,Dog-friendly,travel,Mozilla,Netscape,5.0 (iPhone; CPU iPhone OS 11_3 like Mac OS X)...,...,New,True,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
0000d19da180899b509d8c4560dd9b89,{},"{'REMOTE_ADDR': '34.224.214.253, 35.241.60.129'}",2018-06-25T13:51:41.488Z,06/25/2018 13:51:41,others,others,others,Mozilla,Netscape,5.0 (Macintosh; Intel Mac OS X 10_13_4) AppleW...,...,True,True,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
00050d2cdf8bd5c0e9f0e7aa81d20bf2,{},"{'REMOTE_ADDR': '66.249.69.98, 35.241.60.129'}",2018-07-02T00:00:00.840Z,07/01/2018 17:00:00,others,Shop,pet,Mozilla,Netscape,"5.0 AppleWebKit/537.36 (KHTML, like Gecko; com...",...,True,True,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
00062a5a6bf0e73f7a2c19f4683c7fcf,{},"{'REMOTE_ADDR': '86.14.8.238, 35.241.60.129'}",2018-06-26T05:15:15.273Z2018-06-26T05:14:21.389Z,06/26/2018 06:14:2106/26/2018 06:15:15,travel,Dog-friendly,travel,Mozilla,Netscape,5.0 (Linux; Android 8.1.0; Pixel 2 Build/OPM2....,...,New,True,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
00079523ef5c0c723f455c2e54df5467,{},"{'REMOTE_ADDR': '157.37.91.210, 35.241.60.129'}",2018-06-29T18:33:57.337Z2018-06-29T18:35:10.392Z,06/30/2018 00:03:5706/30/2018 00:05:10,others,others,others,Mozilla,Netscape,5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36...,...,New,True,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
0008639794c055991be9bb1b229786f7,"{}{'breeds': ['Staffordshire Bull Terrier', 'S...","{'REMOTE_ADDR': '82.132.222.44, 35.241.60.129'...",2018-06-30T14:45:01.627Z2018-06-30T14:45:14.99...,06/30/2018 15:45:1506/30/2018 15:46:0506/27/20...,travel,Dog-friendly,travel,Mozilla,Netscape,5.0 (Linux; Android 7.0; SAMSUNG SM-G928F Buil...,...,New,True,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
000a5248b9cd1cf552ca342216b59afa,{},"{'REMOTE_ADDR': '66.249.69.98, 35.241.60.129'}",2018-05-06T00:00:00.866Z,05/05/2018 17:00:00,others,Shop,others,Mozilla,Netscape,"5.0 AppleWebKit/537.36 (KHTML, like Gecko; com...",...,True,True,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
000ac012b90c10fee80998dca96f83e4,{},"{'REMOTE_ADDR': '82.132.222.151, 35.241.60.129'}",2018-07-02T19:52:39.213Z2018-07-02T19:53:48.66...,07/02/2018 20:53:4807/02/2018 20:52:4707/02/20...,travel,Dog-friendly,travel,Mozilla,Netscape,5.0 (Linux; Android 8.0.0; SM-G955F Build/R16N...,...,New,True,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
00162456cee8a0b0b149d0fe32e069e8,{},"{'REMOTE_ADDR': '178.99.135.138, 35.241.60.129'}",2018-07-01T08:25:57.570Z,07/01/2018 09:25:57,others,others,travel,Mozilla,Netscape,5.0 (iPhone; CPU iPhone OS 11_3 like Mac OS X)...,...,True,True,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0


In [17]:

def create_dummy(df, enforce_cols=None):
   
    # create dummy variables for categoricals
   # df.fillna(0, inplace=True)
    
    
    df_all = df[['document.referrer','document.title','href',
            'navigator.appCodeName','navigator.appName',
            'navigator.language','navigator.platform','navigator.product',
            'basket.currency','basket.item_count','basket.coupon',
            'page.location','page.category','page.type','product.category',
            'product.currency','product.url',
            'initial','navigator.cookieEnabled',
            'basket.subtotal','basket.total','baskettotal','basketsubtotal']]
    
    df = pd.get_dummies(df_all,dummy_na = True,
                        columns=['document.referrer','document.title','href',
                                 'navigator.appCodeName','navigator.appName',
                                 'navigator.language','navigator.platform','navigator.product',
                                 'basket.currency','basket.coupon',
                                 'page.location','page.category','page.type','product.category',
                                 'product.currency','product.url',
                                 'initial','navigator.cookieEnabled'])

    
    if enforce_cols is not None:
        to_drop = np.setdiff1d(df.columns, enforce_cols)
        to_add = np.setdiff1d(enforce_cols, df.columns)

        df.drop(to_drop, axis=1, inplace=True)
        df = df.assign(**{c: 0 for c in to_add})
    
    df.fillna(0, inplace=True)
    return df_all,df



In [18]:
df_all,data_set = create_dummy(group_train_pre)
#df_all.dtypes

In [21]:
#data_set.to_csv("data0706_processed.csv")

In [19]:
#df_all.to_csv("data_kmeans.csv")
df_all.to_csv("data_for_clustering.csv")

In [16]:
df_all

,document.referrer,document.title,href,navigator.appCodeName,navigator.appName,navigator.language,navigator.platform,navigator.product,basket.currency,basket.item_count,...,page.type,product.category,product.currency,product.url,initial,navigator.cookieEnabled,basket.subtotal,basket.total,baskettotal,basketsubtotal
MediaGammaImpression,,,,,,,,,,,,,,,,,,,,,
000a5248b9cd1cf552ca342216b59afa,others,Shop,others,Mozilla,Netscape,EN-US,Linux,Gecko,not_pay,0.0,...,,,,others,True,True,NaN,NaN,0.0,0.0
0019ac2bfa1ad975d5c0ccedf09aa9cd,others,others,travel,Mozilla,Netscape,EN-US,iPhone,Gecko,not_pay,0.0,...,,,,others,True,True,NaN,NaN,0.0,0.0
00214fe3b586ac311a8ecfd9e6cc08fa,others,Shop,pet,Mozilla,Netscape,EN-GB,iPad,Gecko,not_pay,0.0,...,,,,others,True,True,NaN,NaN,0.0,0.0
003f2ef971d3b4869481d3c69647e98b,others,Shop,pet,Mozilla,Netscape,EN-GB,iPhone,Gecko,not_pay,0.0,...,,,,others,True,True,NaN,NaN,0.0,0.0
004f2aa1dd8aa80b0a78334aacdabd18,others,Dog-friendly,travel,Mozilla,Netscape,EN-US,Linux,Gecko,not_pay,0.0,...,,,,others,True,True,NaN,NaN,0.0,0.0
005465d3f524adcaa18169808aced682,others,Dog-friendly,travel,Mozilla,Netscape,EN-GB,iPhone,Gecko,not_pay,0.0,...,,,,others,True,True,NaN,NaN,0.0,0.0
00673c3572e23dc39e28d201a894fe09,others,Dog-friendly,travel,Mozilla,Netscape,RO-RO,iPhone,Gecko,not_pay,0.0,...,,,,others,True,True,NaN,NaN,0.0,0.0
007db37b2751551a318e4a03066c4bd8,others,Shop,pet,Mozilla,Netscape,EN-US,Linux,Gecko,not_pay,0.0,...,,,,others,True,True,NaN,NaN,0.0,0.0
00c215a17192c6a21ac936d1155259da,others,others,others,Mozilla,Netscape,EN-GB,Linux,Gecko,not_pay,0.0,...,,,,others,True,True,NaN,NaN,0.0,0.0


In [37]:
df_all.reset_index(level=['MediaGammaImpression'])

,MediaGammaImpression,document.referrer,document.title,href,navigator.appCodeName,navigator.appName,navigator.language,navigator.platform,navigator.product,basket.currency,...,page.type,product.category,product.currency,product.url,initial,navigator.cookieEnabled,basket.subtotal,basket.total,baskettotal,basketsubtotal
0,000a5248b9cd1cf552ca342216b59afa,others,others,others,Mozilla,Netscape,EN-US,Linux,Gecko,not_pay,...,,,,others,True,True,NaN,NaN,0.0,0.0
1,0019ac2bfa1ad975d5c0ccedf09aa9cd,others,others,travel,Mozilla,Netscape,EN-US,iPhone,Gecko,not_pay,...,,,,others,True,True,NaN,NaN,0.0,0.0
2,00214fe3b586ac311a8ecfd9e6cc08fa,others,others,pet,Mozilla,Netscape,EN-GB,iPad,Gecko,not_pay,...,,,,others,True,True,NaN,NaN,0.0,0.0
3,003f2ef971d3b4869481d3c69647e98b,others,others,pet,Mozilla,Netscape,EN-GB,iPhone,Gecko,not_pay,...,,,,others,True,True,NaN,NaN,0.0,0.0
4,004f2aa1dd8aa80b0a78334aacdabd18,others,others,travel,Mozilla,Netscape,EN-US,Linux,Gecko,not_pay,...,,,,others,True,True,NaN,NaN,0.0,0.0
5,005465d3f524adcaa18169808aced682,others,others,travel,Mozilla,Netscape,EN-GB,iPhone,Gecko,not_pay,...,,,,others,True,True,NaN,NaN,0.0,0.0
6,00673c3572e23dc39e28d201a894fe09,others,others,travel,Mozilla,Netscape,RO-RO,iPhone,Gecko,not_pay,...,,,,others,True,True,NaN,NaN,0.0,0.0
7,007db37b2751551a318e4a03066c4bd8,others,others,pet,Mozilla,Netscape,EN-US,Linux,Gecko,not_pay,...,,,,others,True,True,NaN,NaN,0.0,0.0
8,00c215a17192c6a21ac936d1155259da,others,others,others,Mozilla,Netscape,EN-GB,Linux,Gecko,not_pay,...,,,,others,True,True,NaN,NaN,0.0,0.0
9,00f0522ced059966929d2d8d1b9b384b,travel,others,travel,Mozilla,Netscape,EN-GB,iPhone,Gecko,not_pay,...,,,,others,True,True,NaN,NaN,0.0,0.0
